In [1]:
import http.client
import json
import datetime
import pandas as pd
from statistics import mean

In [2]:
def getMonday(dateNow):
    today = dateNow
    today = today + datetime.timedelta(days=-today.weekday(), weeks=-1)
    return today.strftime("%d-%m-%Y")

In [11]:
def getOpenCovidPeruData(date):
    mondayDate = getMonday(date)
    conn = http.client.HTTPSConnection("open-covid-api-vwgk4ckqbq-uk.a.run.app")
    payload = ''
    headers = {}
    conn.request("GET", f"/api/semaforo?fecha={mondayDate}", payload, headers)
    res = conn.getresponse()
    data = res.read()
    jsonData = json.loads(data.decode('utf-8'))
    filteredData =  list(map((lambda value: {'region':value['region'],'appliedTests':value['avgTest'],'testPositivityPercentage':value['positividad'],'population':value['poblacion'],'bedsOccupiedPercentage':value['camasCovid'],'bedsUCIOccupiedPercentage':value['uci'],'confirmedCasesWeekly':(value['poblacion']*value['incid_100'])/100000,'deathsCasesWeekly':(value['poblacion']*value['fall_100'])/100000}), jsonData[0]['regions']))
    df = pd.DataFrame(filteredData)
    return df

def getProcessedTable(date):
    data = getOpenCovidPeruData(date)
    regions = data['region']
    confirmedDeaths100k = (data['deathsCasesWeekly'] / data['population']) * 10**6
    confirmedCases100k = (data['confirmedCasesWeekly'] / data['population']) * 10**6
    appliedTests100k = (data['appliedTests'] / data['population']) * 10**6
    positivityRate = (data['testPositivityPercentage'])
    bedsOccupiedPercentage = (data['bedsOccupiedPercentage'])
    bedsUCIOccupiedPercentage = data['bedsUCIOccupiedPercentage']
    finalDF = pd.concat([regions,confirmedDeaths100k,confirmedCases100k,appliedTests100k,positivityRate,bedsOccupiedPercentage,bedsUCIOccupiedPercentage],axis=1,join='inner',keys=['regions','confirmedDeaths100k', 'confirmedCases100k','appliedTests100k','positivityRate','bedsOccupiedPercentage','bedsUCIOccupiedPercentage'])
    finalDF = finalDF.round(2)
    return finalDF

getProcessedTable(datetime.date.today()-datetime.timedelta(days=9))

,regions,confirmedDeaths100k,confirmedCases100k,appliedTests100k,positivityRate,bedsOccupiedPercentage,bedsUCIOccupiedPercentage
0,AMAZONAS,21.1,133.2,451.44,6.39,9.09,26.32
1,ANCASH,4.8,139.6,412.77,2.73,16.51,68.33
2,APURIMAC,13.0,95.6,592.77,3.42,6.81,20.51
3,AREQUIPA,8.4,172.2,940.50,1.19,42.51,92.22
4,AYACUCHO,8.7,256.5,444.59,8.78,10.53,55.00
5,CAJAMARCA,4.4,105.4,262.17,2.52,3.46,17.46
6,CALLAO,6.1,189.0,435.96,3.76,64.27,75.63
7,CUSCO,8.2,184.8,379.82,3.32,20.34,78.43
8,HUANCAVELICA,18.4,43.7,736.18,1.28,5.88,0.00
9,HUANUCO,6.9,130.5,356.70,4.22,23.66,65.22
